# IR VS IS Pipeline

#### Pipeline Parameters

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from scipy.stats import mannwhitneyu
from statsmodels.stats.multitest import multipletests
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import mutual_info_classif
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn import metrics
from statistics import mode
import openpyxl
import tqdm

import sys 
import joblib
import pickle
import itertools

sys.path.append('H:/Documents/PhD/3rd-year-project/classify-mosquitoes/src/')
import extract, split, config

np.random.seed(0)

In [ ]:
# Segment Size and Overlap (in seconds)
segment_size = 7.5
segment_overlap = 6.5

# Trials split between test/train and validation set
test_trials = np.array([2,3 ,6,7,8, 11,12, 15,16])
target_trials = np.array([1,1, 0,0,0, 0,0, 1,1])
hyp_trials = np.array([0,1, 4,5, 9,10, 13,14])

# Paths
results_path = config.PATH + 'tuned model/random-forests/' # Results stored
data_path = results_path + 'data/' # Any data 

#### Feature Extraction

##### Extract

In [ ]:
tracks, trackTargets, tracksTrialId = extract.load(config.FILE, config.PATH, config.IS_RESISTANT, config.DATA_PATH)

with open(data_path + 'raw_tracks.npy', 'wb') as w:
    np.save(w, np.array(tracks, dtype=object))
with open(data_path + 'raw_trackTargets.npy', 'wb') as w:
    np.save(w, np.array(trackTargets, dtype=object))
with open(data_path + 'raw_tracksTrialId.npy', 'wb') as w:
    np.save(w, np.array(tracksTrialId, dtype=object))

In [ ]:
tracks = np.load(data_path + 'raw_tracks.npy', allow_pickle=True)
trackTargets = np.load(data_path + 'raw_trackTargets.npy', allow_pickle=True)
tracksTrialId = np.load(data_path + 'raw_tracksTrialId.npy', allow_pickle=True)   

tracks = extract.generate_features(tracks, (0,1), 2)

with open(data_path + 'tracks_features.npy', 'wb') as w:
    np.save(w, tracks)

In [ ]:
tracks = np.load(data_path + 'tracks_features.npy', allow_pickle=True)
bmodes = joblib.load('bmodess.dat')
track_id = 0
while track_id < len(tracks):
    mask = np.isin(tracks[track_id][:, 16], bmodes[track_id][: ,0])
    tracks[track_id] = np.insert(tracks[track_id], len(tracks[track_id][0]), mask, axis=1)
    track_id += 1

with open(data_path + 'tracks_features_gaps_marked.npy', 'wb') as w:
    np.save(w, np.array(tracks, dtype=object))

In [ ]:
tracks = np.load(data_path + 'tracks_features_gaps_marked.npy', allow_pickle=True)
trackTargets = np.load(data_path + 'raw_trackTargets.npy', allow_pickle=True)
tracksTrialId = np.load(data_path + 'raw_tracksTrialId.npy', allow_pickle=True)  

tracks, trackTargets, tracksTrialId, trackGroup = split.split_tracks(tracks, trackTargets, tracksTrialId, segment_size, segment_overlap)

In [ ]:
with open(data_path + 'tracks_split.npy', 'wb') as w:
    np.save(w, tracks)
with open(data_path + 'trackTargets_split.npy', 'wb') as w:
    np.save(w, trackTargets)
with open(data_path + 'trackGroup_split.npy', 'wb') as w:
    np.save(w, trackGroup)
with open(data_path + 'tracksTrialId_split.npy', 'wb') as w:
    np.save(w, tracksTrialId)

In [ ]:
tracks = np.load(data_path + 'tracks_split.npy', allow_pickle=True)
trackTargets = np.load(data_path + 'trackTargets_split.npy', allow_pickle=True)
trackGroup = np.load(data_path + 'trackGroup_split.npy', allow_pickle=True)  
tracksTrialId = np.load(data_path + 'tracksTrialId_split.npy', allow_pickle=True)  

In [ ]:
feature_columns = [
    'X Velocity',
    'Y Velocity',
    'X Acceleration', 
    'Y Acceleration',
    'Velocity',
    'Acceleration',
    'Jerk',
    'Angular Velocity',
    'Angular Acceleration',
    'Angle of Flight',
    'Centroid Distance Function',
    'Persistence Velocity',
    'Turning Velocity'
]   
indexes = [12,13,14,15,3,10,17,4,11,18,19,20,21]
feature_stats = [
    'mean','median','std', '1st quartile','3rd quartile','kurtosis', 'skewness',
    'number of local minima','number of local maxima','number of zero-crossings']     

track_statistics = dict()

for col in feature_columns:
    for stat in feature_stats:
        track_statistics[f'{col} ({stat})'] = []

for track in tracks:
    data = extract.track_stats(track, indexes=indexes, columns=feature_columns)
    for d in data:
        track_statistics[d].append(data[d])

df = pd.DataFrame(data=track_statistics)
to_add = extract.add_other_features(tracks, (0,1))
df = pd.concat([df, to_add], axis=1)

df = df.join(pd.DataFrame({'TrialID': tracksTrialId}))

df_target = pd.DataFrame({
    'Target': trackTargets, 'TrialID': tracksTrialId, 'TrackGroup': trackGroup})

In [ ]:
df.to_pickle(data_path + 'df_raw.pkl')
df_target.to_pickle(data_path + 'df_target_raw.pkl')

In [ ]:
indexes = df[df.isna().any(axis=1)].index
df = df.drop(index=indexes)
df_target = df_target.drop(index=indexes)

In [ ]:
df.to_pickle(data_path + 'df_clean.pkl')
df_target.to_pickle(data_path + 'df_target_clean.pkl')

##### Segment Quality Score

In [ ]:
df = pd.read_pickle(data_path + 'df_raw.pkl')
df_target = pd.read_pickle(data_path + 'df_target_raw.pkl')
#tracks = np.load(data_path + 'tracks_split.npy', allow_pickle=True)

In [ ]:
def penalty_function(segment, n, m):
    penalty_score = 0
    k = 0

    for position in segment:
        if position == 0:
            penalty_score += n * (m ** k)
            k += 1
        else:
            k = max(0, k-1)

    return penalty_score/len(segment)

scores = []
for segment in tracks:
    mask = segment[:, -1]
    scores.append(penalty_function(mask, n=1, m=1.05))
scores = np.array(scores)
joblib.dump(scores, data_path+'scores.dat')

In [ ]:
scores = joblib.load(data_path+'scores.dat')

###### Using maximum mutual information to obtain threshold

In [ ]:
def run_score_threshold_mutual_info(df, df_target, scores):
    score_thresholds = np.linspace(0, max(scores), 250)
    max_mutual_info_dict = {}
    unique_features = []
    for threshold in tqdm.tqdm(score_thresholds):
        mask = np.where(scores <= threshold)[0]
        df_temp = df.iloc[mask]
        df_target_temp = df_target.iloc[mask]

        indexes = df_temp[df_temp.isna().any(axis=1)].index
        df_temp = df_temp.drop(index=indexes)
        df_target_temp = df_target_temp.drop(index=indexes)

        df_temp = extract.remove_nans(df_temp)        

        df_temp = df_temp.drop(columns=['TrialID'])
        df_target_temp = df_target_temp['Target']

        mutual_info_values = mutual_info_classif(df_temp, df_target_temp)

        unique_features += df_temp.columns.values.tolist()

        for feature, mutual_info_value in zip(df_temp.columns, mutual_info_values):
            if feature not in max_mutual_info_dict or max_mutual_info_dict[feature]['mutual_info'] < mutual_info_value:
                max_mutual_info_dict[feature] = {
                    'mutual_info': mutual_info_value,
                    'score_threshold': threshold
                }
             
            elif max_mutual_info_dict[feature]['mutual_info'] == mutual_info_value and max_mutual_info_dict[feature]['score_threshold'] < threshold:
                max_mutual_info_dict[feature] = {
                    'mutual_info': mutual_info_value,
                    'score_threshold': threshold
                }

    unique_features = list(set(unique_features))
    corresponding_values = [max_mutual_info_dict[feature]['score_threshold'] for feature in unique_features]

    max_values = [max_mutual_info_dict[feature]['mutual_info'] for feature in unique_features]
    exp = np.exp(np.array(max_values)/(np.array(corresponding_values)+1))
    weights = exp / np.sum(exp)

    weighted_average_threshold = np.average(corresponding_values, weights=weights)

    return weighted_average_threshold

In [ ]:
score_threshold = run_score_threshold_mutual_info(
    df[df['TrialID'].isin(hyp_trials)],
    df_target[df_target['TrialID'].isin(hyp_trials)], 
    scores[df_target['TrialID'].isin(hyp_trials)])

In [ ]:
score_threshold

In [ ]:
mask = np.where(scores <= score_threshold)[0]
df = df.iloc[mask]
df_target = df_target.iloc[mask]

indexes = df[df.isna().any(axis=1)].index
df = df.drop(index=indexes)
df_target = df_target.drop(index=indexes)

df = extract.remove_nans(df)  

In [ ]:
df.to_pickle(data_path + 'df_filtered.pkl')
df_target.to_pickle(data_path + 'df_target_filtered.pkl')

###### Using gradient of mutual information curves to obtain threshold

In [ ]:
def calculate_gradient(data, window_size):
    gradients = np.gradient(data)
    smoothed_gradients = np.convolve(gradients, np.ones(window_size)/window_size, mode='valid')
    return smoothed_gradients

def find_gradient(data, threshold=0.1, window_size=3):
    gradients = calculate_gradient(data, window_size)
    total_change = data[-1] - data[0]
    threshold_value = threshold * total_change
    for i, gradient in enumerate(gradients):
        if gradient < threshold_value:
            return i + window_size // 2 
    return i

def run_score_threshold_gradient(df, df_target, scores):
    score_thresholds = np.linspace(0, max(scores), 250)

    unique_features = []
    all_scores_xls = []
    for threshold in score_thresholds:
        mask = np.where(scores <= threshold)[0]
        df_temp = df.iloc[mask]
        df_target_temp = df_target.iloc[mask]

        indexes = df_temp[df_temp.isna().any(axis=1)].index
        df_temp = df_temp.drop(index=indexes)
        df_target_temp = df_target_temp.drop(index=indexes)

        df_temp = extract.remove_nans(df_temp)        

        df_temp = df_temp.drop(columns=['TrialID'])
        df_target_temp = df_target_temp['Target']

        mutual_info_values = mutual_info_classif(df_temp, df_target_temp)

        unique_features += df_temp.columns.values.tolist()

        for feature, mutual_info_value in zip(df_temp.columns.values, mutual_info_values):
            all_scores_xls.append({'feature': feature, 'mutual_info': mutual_info_value, 'threshold': threshold})
            
    d = pd.DataFrame(all_scores_xls)
    d = d.pivot(index='threshold', columns='feature', values='mutual_info')
    d.reset_index(inplace=True)

    feature_thresholds = {}
    for feature in d.columns.values:
        if feature != 'threshold':
            index = find_gradient(d[feature].values)
            feature_thresholds[feature] = {'threshold': d['threshold'].iloc[index], 'mutual_info': d[feature].iloc[index]}

    unique_features = list(set(unique_features))
    corresponding_values = [feature_thresholds[feature]['threshold'] for feature in unique_features]

    max_values = [feature_thresholds[feature]['mutual_info'] for feature in unique_features]
    exp = np.exp(max_values)
    weights = exp / np.sum(exp)

    weighted_average_threshold = np.average(corresponding_values, weights=weights)

    return weighted_average_threshold

In [ ]:
score_threshold = run_score_threshold_gradient(    
    df[df['TrialID'].isin(hyp_trials)],
    df_target[df_target['TrialID'].isin(hyp_trials)], 
    scores[df_target['TrialID'].isin(hyp_trials)])

In [ ]:
mask = np.where(scores <= score_threshold)[0]
df = df.iloc[mask]
df_target = df_target.iloc[mask]

indexes = df[df.isna().any(axis=1)].index
df = df.drop(index=indexes)
df_target = df_target.drop(index=indexes)

df = extract.remove_nans(df)  

In [ ]:
df.to_pickle(data_path + 'df_filtered.pkl')
df_target.to_pickle(data_path + 'df_target_filtered.pkl')

#### Split Train-Test/Validation sets

In [ ]:
df = pd.read_pickle(data_path + 'df_filtered.pkl')
df_target = pd.read_pickle(data_path + 'df_target_filtered.pkl')

In [ ]:
df_train = df[df['TrialID'].isin(test_trials)]
df_train_target = df_target[df_target['TrialID'].isin(test_trials)]

df_hyp = df[df['TrialID'].isin(hyp_trials)]
df_hyp_target = df_target[df_target['TrialID'].isin(hyp_trials)]

In [ ]:
df_hyp.to_pickle(data_path + 'df_hyp.pkl')
df_hyp_target.to_pickle(data_path + 'df_hyp_target.pkl')

#### Feature Selection

In [ ]:
df_hyp = df_hyp.drop(columns=['TrialID'])

sus = df_hyp[df_hyp_target['Target'] == 0]
res = df_hyp[df_hyp_target['Target'] == 1]
_, p_val = mannwhitneyu(sus, res)
rej, p_vals_corrected, _, _ = multipletests(p_val, alpha=0.05, method='holm')

columns = df_hyp.columns[rej]

df_hyp = df_hyp.reset_index(drop=True)
corr_matrix = df_hyp.corr(method='spearman').abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.85)]

cols = np.setdiff1d(columns, to_drop)

In [ ]:
features = []
for feat in cols:
    if 'TrialID' not in feat:
        features.append(feat)

In [ ]:
file = open(results_path + 'features.txt', 'w+')
features = []
for feat in cols:
    if 'TrialID' not in feat:
        features.append(feat)
        file.write(feat+'\n')

file.close()

In [ ]:
features = open(results_path + 'features.txt', 'r+').read().split('\n')
features.remove('')

In [ ]:
len(features)

In [ ]:
for i in sorted(features):
    print(i)

## Model

#### Create Folds

In [ ]:
def create_folds():
    final_folds = []
    for v1 in list(itertools.product([2,3], [15,16])):
        for v2 in list(itertools.product([6,7,8], [11,12])):
            train_trials = list(v1) + list(v2)
            final_folds.append(train_trials)
    return final_folds

In [ ]:
a = create_folds()
file = open(results_path+'folds.txt', 'w+')
for f in a:
    file.write(str(f) +'\n')
file.close()

#### Model Building

In [ ]:
def perf_measure(y_actual, y_hat):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_hat)): 
        if y_actual[i]==y_hat[i]==1:
           TP += 1
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
           FP += 1
        if y_actual[i]==y_hat[i]==0:
           TN += 1
        if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
           FN += 1

    return TP, FP, TN, FN


def get_track_prediction(y_true, scores, preds, groups):
    unique_groups = groups.unique()
    track_preds = []
    track_true = []
    avg_scores = []
    for val in unique_groups:
        indexes = np.where(groups == val)[0]
        track_true.append(mode(y_true.values[indexes]))
        avg_scores.append(np.mean(scores[indexes]))
        if np.mean(preds[indexes]) >= 0.5: 
            track_preds.append(1)
        else:
            track_preds.append(0)

    return track_true, track_preds, avg_scores


def produce_report(y_test, y_pred, scores):
    precision_ir, recall_ir, _ = metrics.precision_recall_curve(y_test, scores)
    precision_is, recall_is, _ = metrics.precision_recall_curve((np.array(y_test)*-1)+1 , 1-np.array(scores))
    
    TP, FP, TN, FN = perf_measure(y_test, y_pred)
    fpr, tpr, _ = metrics.roc_curve(y_test, scores)
    data = {
        'accuracy': metrics.accuracy_score(y_test,y_pred),
        'balanced accuracy': metrics.balanced_accuracy_score(y_test,y_pred),
        'roc auc': metrics.roc_auc_score(y_test, scores),

        'f1 score (ir)': metrics.f1_score(y_test,y_pred),
        'precision (ir)': metrics.precision_score(y_test, y_pred),
        'recall (ir)': metrics.recall_score(y_test, y_pred),
        'pr auc (ir)': metrics.auc(recall_ir, precision_ir),

        'f1 score (is)': metrics.f1_score(np.array(y_test)*-1 +1, np.array(y_pred)*-1 +1),
        'precision (is)': metrics.precision_score(np.array(y_test)*-1 +1, np.array(y_pred)*-1 +1),
        'recall (is)': metrics.recall_score(np.array(y_test)*-1 +1, np.array(y_pred)*-1 +1),
        'pr auc (is)': metrics.auc(recall_is, precision_is),

        'cohen kappa': metrics.cohen_kappa_score(np.array(y_test), np.array(y_pred)),
        'matthews correlation coefficient': metrics.matthews_corrcoef(np.array(y_test), np.array(y_pred)),
        'log loss': metrics.log_loss(np.array(y_test), np.array(scores)),
        'tp': TP, 'tn': TN, 'fp': FP, 'fn': FN,
        'tpr': tpr, 'fpr': fpr
    }
    return data

In [ ]:
def constant_model(x_train, y_train, x_test, y_test):
    model = DummyRegressor(strategy='constant', constant=0)
    model.fit(x_train, y_train['Target'])
    y_pred = model.predict(x_test)
    track_true, track_preds, scores = get_track_prediction(y_test['Target'], y_pred, y_pred, y_test['TrackGroup'])
    return produce_report(track_true, track_preds, scores)  , {
        'segment-preds': y_pred, 
        'segment-target': y_test['Target']
    }  

def logistic_model(x_train, y_train, x_test, y_test, params):
    model = LogisticRegression(
        random_state=0,
        **params

    )
    model.fit(x_train, y_train['Target'])
    scores = model.predict_proba(x_test)[:,1]
    y_pred = model.predict(x_test)
    track_true, track_preds, avg_scores = get_track_prediction(y_test['Target'], scores, y_pred, y_test['TrackGroup'])
    return produce_report(track_true, track_preds, avg_scores), {
        'segment-preds': y_pred, 
        'segment-target': y_test['Target']
    }, model

def random_forest_model(x_train, y_train, x_test, y_test, params):
    model = RandomForestClassifier(
        **params
    )
    model.fit(x_train, y_train['Target'])
    y_pred = model.predict(x_test)
    scores = model.predict_proba(x_test)[:,1]
    track_true, track_preds, avg_scores = get_track_prediction(y_test['Target'], scores, y_pred, y_test['TrackGroup'])
    return produce_report(track_true, track_preds, avg_scores), {
        'segment-preds': y_pred, 
        'segment-target': y_test['Target']
    }, model

def xgboost_model(x_train, y_train, x_test, y_test, params):
    model = XGBClassifier(
        **params
    )
    model.fit(x_train, y_train['Target'])
    y_pred = model.predict(x_test)
    scores = model.predict_proba(x_test)[:,1]
    track_true, track_preds, avg_scores = get_track_prediction(y_test['Target'], scores, y_pred, y_test['TrackGroup'])
    return produce_report(track_true, track_preds, avg_scores), {
        'segment-preds': y_pred, 
        'segment-target': y_test['Target']
    }, model

In [ ]:
results = dict(
    dummy_results_train = [],
    dummy_results_test = [],
    logistic_results_train = [],
    logistic_results_test = [],
    random_forests_train = [],
    random_forests_test = [],
    xgboost_train = [],
    xgboost_test = [],
)
segment_scores_train = []
segment_scores_test = []

segment_scores_lr_train = []
segment_scores_lr_test = []

segment_scores_constant_train = []
segment_scores_constant_test = []

segment_scores_xgboost_train = []
segment_scores_xgboost_test = []

folds = create_folds()


for index, fold in enumerate(folds):
    print(f' --- FOLD {index} ---')
    train_trials = fold
    mask = df_train_target['TrialID'].isin(train_trials)

    train = df_train[mask]
    train_targets = df_train_target[mask]
    
    test = df_train[~mask]
    test_targets = df_train_target[~mask]

    scaler = StandardScaler()
    train = pd.DataFrame(scaler.fit_transform(train[features]), columns=features, index=train.index)
    test = pd.DataFrame(scaler.transform(test[features]), columns=features, index=test.index)

    sm = SMOTE(
        random_state=0
    )
    train_os, train_targets_os = sm.fit_resample(train, train_targets.drop(columns=['TrialID','TrackGroup']))
    
    constant_scores, constant_segment_scores = constant_model(
        x_train=train_os, 
        y_train=train_targets_os, 
        x_test=train, 
        y_test=train_targets)
    results['dummy_results_train'].append(constant_scores)
    segment_scores_constant_train.append(constant_segment_scores)

    constant_scores, constant_segment_scores = constant_model(
        x_train=train_os, 
        y_train=train_targets_os, 
        x_test=test, 
        y_test=test_targets)

    results['dummy_results_test'].append(constant_scores)
    segment_scores_constant_test.append(constant_segment_scores)
    
    
    lr_scores, lr_segment_scores, model = logistic_model(
        x_train=train_os, 
        y_train=train_targets_os, 
        x_test=train, 
        y_test=train_targets,
        params=dict(
            penalty='l2',
            C=0.01,
            solver='saga',
            max_iter=50
        )
    )
    results['logistic_results_train'].append(lr_scores)
    segment_scores_lr_train.append(lr_segment_scores)
    
    lr_scores, lr_segment_scores, model = logistic_model(
        x_train=train_os, 
        y_train=train_targets_os, 
        x_test=test, 
        y_test=test_targets,
        params=dict(
            penalty='l2',
            C=0.01,
            solver='saga',
            max_iter=50
        )
    )
    results['logistic_results_test'].append(lr_scores)
    segment_scores_lr_test.append(lr_segment_scores)
    joblib.dump(dict(
        model=model,
        df_train=df_train,
        df_train_target=df_train_target,
        features=features,
        train_os=train_os,
        test=test,
        mask=mask,
    ), data_path+f'shap/logistic_shap_dump_{index}.dat')
    
     
    rf_scores, segment_scores, model = random_forest_model(
        x_train=train_os, 
        y_train=train_targets_os, 
        x_test=train, 
        y_test=train_targets,
        params=dict(
            random_state=0,
            n_estimators=300,
            criterion='entropy',
            max_depth=15,
            min_samples_split=3,
            min_samples_leaf=1,
            max_features='sqrt',
            bootstrap=False,
            n_jobs=8
        ))
    results['random_forests_train'].append(rf_scores)
    segment_scores_train.append(segment_scores)

    rf_scores, segment_scores, model = random_forest_model(
        x_train=train_os, 
        y_train=train_targets_os, 
        x_test=test, 
        y_test=test_targets,
        params=dict(
            random_state=0,
            n_estimators=300,
            criterion='entropy',
            max_depth=15,
            min_samples_split=3,
            min_samples_leaf=1,
            max_features='sqrt',
            bootstrap=False,
            n_jobs=8
        ))
    results['random_forests_test'].append(rf_scores)
    segment_scores_test.append(segment_scores)
    joblib.dump(dict(
        model=model,
        df_train=df_train,
        df_train_target=df_train_target,
        features=features,
        test=test,
        mask=mask,
        train_os=train_os,
    ), data_path+f'shap/random_forests_shap_dump_{index}.dat')
     
    
    xg_scores, segment_scores, model = xgboost_model(
        x_train=train_os, 
        y_train=train_targets_os, 
        x_test=train, 
        y_test=train_targets,
        params=dict(
            random_state=0,
            learning_rate=0.3,
            n_estimators=250,
            max_depth=5,
            subsample=0.5,
            colsample_bytree=0.9,
            reg_alpha=0, 
            reg_lambda=0,
            min_child_weight=15
        ))
    results['xgboost_train'].append(xg_scores)
    segment_scores_xgboost_train.append(segment_scores)

    xg_scores, segment_scores, model = xgboost_model(
        x_train=train_os, 
        y_train=train_targets_os, 
        x_test=test, 
        y_test=test_targets,
        params=dict(
            random_state=0,
            learning_rate=0.3,
            n_estimators=250,
            max_depth=5,
            subsample=0.5,
            colsample_bytree=0.9,
            reg_alpha=0, 
            reg_lambda=0,
            min_child_weight=15
        ))
    results['xgboost_test'].append(xg_scores)
    segment_scores_xgboost_test.append(segment_scores)
    joblib.dump(dict(
        model=model,
        df_train=df_train,
        df_train_target=df_train_target,
        features=features,
        test=test,
        test_targets=test_targets,
        mask=mask,
        train_os=train_os,
    ), data_path+f'shap-1/xgboost_shap_dump_{index}.dat')

In [ ]:
with open(results_path+'results.pkl', 'wb') as f:
    pickle.dump(results, f)

In [ ]:
with open(results_path+'results.pkl', 'rb') as f:
    results = pickle.load(f)

#### Model Performance

In [ ]:
wb = openpyxl.Workbook()
sheet = wb.create_sheet()

row = 2
metrics_list = list(results['random_forests_train'][0].keys())
for i, column in enumerate(['model'] + metrics_list):
    sheet.cell(row=1, column=i+1).value = column

for model in ['dummy_results', 'logistic_results', 'random_forests', 'xgboost']:
    for model_type in ['train', 'test']:
        try:
            sheet.cell(row=row, column=1).value = f'{model.upper()} {model_type.upper()}'
            for j, metric in enumerate(results[model+'_'+model_type][0].keys()):
                if metric in ['tp', 'tn', 'fp', 'fn']:
                    scores = 0
                    for fold in range(len(results[model+'_'+model_type])):
                        scores += results[model+'_'+model_type][fold][metric]
                    
                    sheet.cell(row=row, column=j+2).value = scores 
                elif metric in ['tpr', 'fpr']:
                    pass
                else:
                    scores = []
                    for fold in range(len(results[model+'_'+model_type])):
                        scores.append(results[model+'_'+model_type][fold][metric])
                    
                    sheet.cell(row=row, column=j+2).value = f'{round(np.mean(scores), 3)} ({round(min(scores), 3)} - {round(max(scores), 3)})'
            row += 1
        except:
            pass

wb.save(results_path + 'scores.xlsx')

#### Graphs

In [ ]:
'''CONFUSION MATRIX'''

def plot_confusion_matrix(tp, tn, fp, fn, classifier):
    confusion_matrix = np.array([[tn, fp], [fn, tp]])
    fig, ax = plt.subplots(dpi=300)
    im = ax.imshow(confusion_matrix, cmap='Oranges')
    ax.set_xlabel('Predicted Class')
    ax.set_ylabel('True Class')
    #ax.set_title(f'Confusion Matrix - {classifier}')

    ax.set_xticks([0, 1])
    ax.set_yticks([0, 1])
    ax.set_xticklabels(['IS', 'IR'])
    ax.set_yticklabels(['IS', 'IR'])

    for i in range(2):
        for j in range(2):
            ax.text(j, i, confusion_matrix[i, j], ha='center', va='center', fontsize=14)

    plt.show()

def plot_confusion_matrix_percent(tp, tn, fp, fn, classifier):
    confusion_matrix = np.array([[tn, fp], [fn, tp]])
    fig, ax = plt.subplots(dpi=300)
    im = ax.imshow(confusion_matrix, cmap='Oranges')
    ax.set_xlabel('Predicted Class')
    ax.set_ylabel('True Class')
    #ax.set_title(f'Confusion Matrix - {classifier}')

    ax.set_xticks([0, 1])
    ax.set_yticks([0, 1])
    ax.set_xticklabels(['IS', 'IR'])
    ax.set_yticklabels(['IS', 'IR'])

    total_samples = np.sum(confusion_matrix)
    for i in range(2):
        for j in range(2):
            percentage = confusion_matrix[i, j] / total_samples * 100
            ax.text(j, i, f'{percentage:.2f}%', ha='center', va='center', color='black', fontsize=14)
    plt.show()


for key in ['dummy_results_train', 'dummy_results_test','logistic_results_train','logistic_results_test','random_forests_train','random_forests_test', 'xgboost_train', 'xgboost_test']:
    print(key)
    plot_confusion_matrix_percent(
        sum([_['tp'] for _ in results[key]]), 
        sum([_['tn'] for _ in results[key]]), 
        sum([_['fp'] for _ in results[key]]), 
        sum([_['fn'] for _ in results[key]]),
        classifier=key
    )

In [ ]:
'''ROC CURVES'''

def plot_roc_curve(tpr, fpr, classifier):
    fig, ax = plt.subplots(dpi=300)
    for i in range(len(tpr)):
        ax.plot(fpr[i], tpr[i], alpha=0.5)

    ax.plot([0, 1], [0, 1], linestyle='--')
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    #ax.set_title(f'ROC Curves - {classifier}')
    plt.show()


for key in ['dummy_results_train', 'dummy_results_test','logistic_results_train','logistic_results_test','random_forests_train','random_forests_test', 'xgboost_train', 'xgboost_test']:
    print(key)
    plot_roc_curve(
        [_['tpr'] for _ in results[key]],  
        [_['fpr'] for _ in results[key]], 
        classifier=key
    )

In [ ]:
'''PR CURVES'''

def plot_pr_curve(precision, recall, classifier):
    fig, ax = plt.subplots()
    base = np.linspace(0, 1, 101)
    avg_precision = 1

    for i in range(len(precision)):
        ax.plot(recall[i], precision[i])
        new_avg_precision = sum(precision[i]) / len(precision[i])
        if new_avg_precision < avg_precision:
            avg_precision = new_avg_precision

    ax.plot(base, [avg_precision]*len(base), linestyle='--')
    ax.set_xlabel('Recall')
    ax.set_ylabel('Precision')
    ax.set_title(f'PR Curves - {classifier}')

    plt.show()


for key in ['dummy_results_train', 'dummy_results_test','logistic_results_train','logistic_results_test','random_forests_train','random_forests_test', 'xgboost_train', 'xgboost_test']:
    plot_pr_curve(
        [results[key][i]['precision (ir) (curve)'] for i in range(len(results[key]))],
        [results[key][i]['recall (ir) (curve)'] for i in range(len(results[key]))], 
        classifier=key+' (ir)'
    )
    plot_pr_curve(
        [results[key][i]['precision (is) (curve)'] for i in range(len(results[key]))],
        [results[key][i]['recall (is) (curve)'] for i in range(len(results[key]))], 
        classifier=key+' (is)'
    )

In [ ]:
'''EXCEL FILE OF ALL FOLD SCORES'''

wb = openpyxl.Workbook()
for key in ['dummy_results_train', 'dummy_results_test','logistic_results_train','logistic_results_test','random_forests_train','random_forests_test', 'xgboost_train', 'xgboost_test']:
    sheet = wb.create_sheet(key.upper())
    columns = ['fold', 'test trials', 'train trials', 'accuracy', 'balanced accuracy',
    'f1 score (ir)', 'roc auc', 'precision (ir)', 'recall (ir)', 'pr auc (ir)',
    'f1 score (is)', 'precision (is)', 'recall (is)', 'pr auc (is)',
    'cohen kappa', 'matthews correlation coefficient', 'log loss', 'tp', 'tn', 'fp', 'fn']
    for i, column in enumerate(columns):
        sheet.cell(row=1, column=i+1).value = column

        for row in range(len(results[key])):
            if column not in ['fold', 'test trials', 'train trials']:
                sheet.cell(row=row+2, column=i+1).value = results[key][row][column]
        
            elif column == 'fold':
                sheet.cell(row=row+2, column=i+1).value = row

            elif column == 'test trials':
                train = folds[row]
                all_ids = df_train_target['TrialID'].unique()
                sheet.cell(row=row+2, column=i+1).value = str([x for x in all_ids if x not in train])

            elif column == 'train trials':
                sheet.cell(row=row+2, column=i+1).value = str(folds[row])

wb.save(results_path + "all-folds.xlsx")